<a href="https://colab.research.google.com/github/Inishpy/DeepLearningWork/blob/main/FlaskAppRunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_socketio
!pip install llama-cpp-python
!pip install piper-tts
!pip install pyaudio

!pip install torch
!pip install transformers
!pip install sounddevice
!pip install pydub
!pip install espeak_phonemizer
!pip install ollama


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 34.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.55-cp310-cp310-manylinux_2_35_x86_64.whl size=2783050 sha256=ca52cf563313f93df9ec8ed3af94a5e406f13f247770d968bc3f7679e5c758e5
  Stored in directory: /root/.cache/pip/wheels/35/c9/bb/019dbfeef119ab5c29f76574b76070afa7b7755ccfbb3ee226
Successfully built llama-cpp-python
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/25.0 MB 9.3 MB/s eta 0:00:00
    

In [ ]:
# %%

import torch

from transformers import pipeline
import time

from llama_cpp import Llama
from huggingface_hub import hf_hub_download
import numpy as np


import requests
#import sounddevice as sd


import wave
#import pyaudio




import threading
import time
import queue
import sys
import re

from queue import Queue
from flask_socketio import SocketIO, emit
#from jetsonGPTmain import webserver

#import ollama


# %%
#pip install torch
#pip install transformers
#!pip install sounddevice
#!pip install pydub
#!pip install espeak_phonemizer

# %%
def print_with_newline(string):
    words = string.split()
    for i in range(0, len(words), 10):
        print(' '.join(words[i:i+10]))

def replace_apostrophes(string):
    return string.replace('"', "'")

# %%
#!pip install llama-cpp-python
#!pip install piper-tts
#!pip install pyaudio

# %%


# %%



#pipe = pipeline("automatic-speech-recognition", "openai/whisper-tiny")
#pipe1 =  pipeline("automatic-speech-recognition", "openai/whisper-base")
#pipe2 = pipeline("automatic-speech-recognition", "openai/whisper-small")
#pipe3 = pipeline("automatic-speech-recognition", "openai/whisper-medium")
#pipe4 = pipeline("automatic-speech-recognition", "openai/whisper-large-v2")
# {'text': "GOING ALONG SLUSHY COUNTRY ROADS AND SPEAKING TO DAMP AUDIENCES IN DRAUGHTY SCHOOL ROOMS DAY AFTER DAY FOR A FORTNIGHT HE'LL HAVE TO PUT IN AN APPEARANCE AT SOME PLACE OF WORSHIP ON SUNDAY MORNING AND HE CAN COME TO US IMMEDIATELY AFTERWARDS"}


# %%






def record_audio(file_path, duration=5, sample_rate=44100):
    print(f"Recording audio for {duration} seconds...")

    # Initialize PyAudio
    audio = pyaudio.PyAudio()

    # Define audio stream parameters
    chunk_size = 1024  # Adjust the chunk size as needed
    format = pyaudio.paInt16
    channels = 2

    # Open audio stream
    stream = audio.open(format=format,
                        channels=channels,
                        rate=sample_rate,
                        input=True,
                        frames_per_buffer=chunk_size)

    frames = []
    # Record audio for the specified duration
    for _ in range(0, int(sample_rate / chunk_size * duration)):
        data = stream.read(chunk_size)


        frames.append(data)


    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    audio.terminate()

    print("Recording complete. Saving to file.")

    array_data = np.frombuffer(b''.join(frames), dtype=np.float16)
    #b  = pipe1({"sampling_rate":  44100, "raw": array_data})['text']
    #print(b)

    # Save audio data to a WAV file
    with wave.open(file_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(audio.get_sample_size(format))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(frames))

    print(f"Audio saved to {file_path}")




# %%
#!pip install torchaudio

# %%


# %%
global_stop_event = threading.Event()

def my_function(thread_lock, thread_id, sentence):
    with thread_lock:

        if not global_stop_event.is_set():

            st = time.time()
            # Read audio data from stdin
            data = webserver.predict(sentence)
            end = time.time()
            #print("script", end - st)
            npa = np.asarray(data['data'], dtype=np.int16)
            sd.play(npa, data['sample-rate'], blocking=True)


def worker(thread_lock, thread_id, sentence_queue):
    while True:
        sentence = sentence_queue.get()
        if sentence is None:
            break
        my_function(thread_lock, thread_id, sentence)
        sentence_queue.task_done()


#model = "./phi-2-dpo.Q5_K_S.gguf"
model = hf_hub_download(repo_id="TheBloke/phi-2-dpo-GGUF", filename="phi-2-dpo.Q5_K_S.gguf")
llm = Llama(model_path=model, chat_format="llama-2" )  # Set chat_format according to the model you are using





def modelRun(textinput=False, input=""):


    Ollama = False
    #record_audio("recorded_audio.wav", duration=6)
    print("recorded")

    if textinput == True:
        QUERY = input
    else:
        record_audio("recorded_audio.wav", duration=6)
        QUERY = pipe1("recorded_audio.wav")['text']
    print(QUERY)


    # Chat Completion API
    thread_lock = threading.Lock()
    sentence_queue = queue.Queue()

    #Create worker threads

    workers = []

    output = """ """
    cur_sentence = ""

    if Ollama == True:
        stream = ollama.chat(
        model='phi',
        messages=[{'role': 'user', 'content': QUERY}],
        stream=True,
    )

        for chunk in stream:

            if global_stop_event.is_set():
                return

            character = chunk['message']['content']
            print(character)
            #emit('sentence', {'message': character})
            output += character
            cur_sentence += character
            #print(character)


            if re.search(r'[.,]$', character):#detect fullstop at the end of the sentence
                # Enqueue the sentence
                sentence_queue.put(cur_sentence)


                cur_sentence = ""
                worker_thread = threading.Thread(target=worker, args=(thread_lock, 1, sentence_queue))
                worker_thread.start()
                workers.append(worker_thread)
    else:

        for token in  (llm.create_chat_completion(
            messages = [
                # {"role": "system", "content": "You are a story writing assistant."},
                {
                    "role": "user",
                    "content": QUERY + "."
                }
            ],
            stream = True,
            max_tokens = 200
        )):
            character = token['choices'][0]['delta'].get('content', "")
            print(character)
            emit('sentence', {'data': character})
            output += character
            """
            cur_sentence += character
            #print(character)


            if re.search(r'[.,]$', character):#detect fullstop at the end of the sentence
                # Enqueue the sentence
                sentence_queue.put(cur_sentence)

                cur_sentence = ""
                worker_thread = threading.Thread(target=worker, args=(thread_lock, 1, sentence_queue))
                worker_thread.start()
                workers.append(worker_thread)

    if cur_sentence != "":
        sentence_queue.put(cur_sentence)
        cur_sentence = ""

        worker_thread = threading.Thread(target=worker, args=(thread_lock, 1, sentence_queue))
        worker_thread.start()
        workers.append(worker_thread)


    # Wait for all sentences to be processed
    sentence_queue.join()

    # Stop worker threads by adding None to the queue
    for _ in range(len(workers)):
        sentence_queue.put(None)

    # Wait for worker threads to finish
    for worker_thread in workers:
        worker_thread.join()





    print("------------------------------------------")
    print_with_newline(output)
    print("------------------------------------------")




"""










In [ ]:
!pip install gevent-websocket

In [ ]:
import os

os.environ['GEVENT_SUPPORT'] = 'True'

In [ ]:
from gevent import monkey
monkey.patch_all()

It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
<ipython-input-6-1bad42606e85>:2: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['aiohttp.client_exceptions (/usr/local/lib/python3.10/dist-packages/aiohttp/client_exceptions.py)', 'urllib3.util (/usr/local/lib/python3.10/dist-packages/urllib3

True

https://w7b04mr1n8-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:


from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))


from flask import Flask, render_template, request
from flask_socketio import SocketIO, emit
import time
app = Flask(__name__, template_folder='/content/drive/MyDrive/Rogerflask/templates')
#import roger
# Replace with a strong secret key
socketio = SocketIO(app, async_mode='gevent')

@app.route('/')
def index():
    return render_template('index.html')


@socketio.on('connect')
def handle_connect():
    print('Client connected!')
    emit('connected', {'message': 'Welcome!'})


@socketio.on('start')
def handle_message(data):
    global_stop_event.clear()
    query = data["input"]
    if query == "":
        modelRun(False)
    else:
        modelRun(True, query)


@socketio.on('stopmodel')
def stop_model():
    print("stop-model-receidved")
    global_stop_event.set()


if __name__ == "__main__":
  app.run()
  #socketio.run(app, allow_unsafe_werkzeug=True)

It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is be

KeyboardInterrupt: 

It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.


In [ ]:
pip install https://github.com/eventlet/eventlet/archive/master.zip

In [ ]:
!gunicorn --worker-class eventlet -w 1 module:app

In [ ]:
!pip install gunicorn
!pip install eventlet
!pip install gevent

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))




In [ ]:
!gunicorn -k gevent -w 1 wsgi:app

In [ ]:
!pip install https://github.com/eventlet/eventlet/archive/master.zip

In [ ]:
!pip install pyngrok

In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
port_no = 5000

In [ ]:
app = Flask(__name__)
ngrok.set_auth_token("2dDg3t3XLJu0CE56k8QnqVsVh7p_67VUuJGu2eE19XGDLHKkJ")
public_url =  ngrok.connect(port_no).public_url

@app.route("/")
def home():
    return f"Running Flask on Google Colab!"

print(f"To acces the Gloable link please click {public_url}")

app.run(port=port_no)

To acces the Gloable link please click https://b6a4-35-190-172-102.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 11:46:23] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2024 11:46:24] "GET /favicon.ico HTTP/1.1" 404 -
